# Генерація великого синтетичного датасету для задачі прогнозування відвідуваності занять

Цей ноутбук створює великий штучний датасет для задачі регресії: прогнозування кількості студентів, які прийдуть на пару, з урахуванням різних факторів.

## 1. Імпорт необхідних бібліотек

Імпортуємо pandas, numpy, random, datetime та інші потрібні бібліотеки для генерації та обробки даних.

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import calendar

## 2. Визначення структури датасету та факторів

Опишемо всі фактори, які будуть враховані у датасеті, та створимо відповідні списки/словники для категоріальних змінних.

In [2]:
# Категоріальні та числові фактори
weekdays = ['Понеділок', 'Вівторок', 'Середа', 'Четвер', 'Пʼятниця', 'Субота']
lesson_times = ['8:30', '10:10', '11:50', '13:30', '15:05', '16:40', '18:10', '19:40']
subject_importance_study = [1, 2, 3, 4, 5]  # 1 - неважливо, 5 - дуже важливо
subject_importance_life = [1, 2, 3, 4, 5]
teacher_strictness = [1, 2, 3, 4, 5]  # 1 - нестрогий, 5 - дуже строгий
attendance_check = [0, 1]  # 0 - не відмічають, 1 - відмічають
weather_rain = [0, 1]  # 0 - без дощу, 1 - дощ
temperature_range = (-10, 35)  # Температура в градусах Цельсія
exam_or_test = [0, 1]  # 0 - ні, 1 - так
prev_day_off = [0, 1]
next_day_off = [0, 1]
special_event = [0, 1]  # 0 - ні, 1 - так (свято, карантин)
weeks_to_exam = list(range(0, 16))  # 0 - сесія, 15 - початок семестру
prev_lesson = [0, 1]  # 0 - не було, 1 - була
next_lesson = [0, 1]
lesson_type = ['Лекція', 'Лабораторна']

## 3. Генерація категоріальних та числових ознак

Згенеруємо випадкові значення для факторів: важливість предмету, строгість викладача, чи відмічають присутніх, контрольна/залік, тип пари тощо.

In [3]:
# Кількість рядків у датасеті
N = 30000

data = []

for _ in range(N):
    weekday = random.choice(weekdays)
    lesson_time = random.choice(lesson_times)
    imp_study = random.choice(subject_importance_study)
    imp_life = random.choice(subject_importance_life)
    strictness = random.choice(teacher_strictness)
    attendance = random.choice(attendance_check)
    rain = random.choice(weather_rain)
    temperature = random.randint(*temperature_range)
    exam = random.choices(exam_or_test, weights=[0.85, 0.15])[0]
    prev_off = random.choice(prev_day_off)
    next_off = random.choice(next_day_off)
    event = random.choices(special_event, weights=[0.92, 0.08])[0]
    weeks_exam = random.choice(weeks_to_exam)
    prev_l = random.choice(prev_lesson)
    next_l = random.choice(next_lesson)
    l_type = random.choices(lesson_type, weights=[0.7, 0.3])[0]
    data.append([
        weekday, lesson_time, imp_study, imp_life, strictness, attendance, rain, temperature,
        exam, prev_off, next_off, event, weeks_exam, prev_l, next_l, l_type
    ])

columns = [
    'День_тижня', 'Час_пари', 'Важливість_у_навчанні', 'Важливість_у_житті', 'Строгість_викладача',
    'Відмічають_присутність', 'Дощ', 'Температура', 'Контрольна_або_залік', 'Вихідний_напередодні',
    'Вихідний_завтра', 'Подія', 'Тижнів_до_екзамену', 'Попередня_пара', 'Наступна_пара',
    'Тип_пари'
]
df = pd.DataFrame(data, columns=columns)
df.head()

,День_тижня,Час_пари,Важливість_у_навчанні,Важливість_у_житті,Строгість_викладача,Відмічають_присутність,Дощ,Температура,Контрольна_або_залік,Вихідний_напередодні,Вихідний_завтра,Подія,Тижнів_до_екзамену,Попередня_пара,Наступна_пара,Тип_пари
0,Понеділок,18:10,4,5,5,1,1,30,0,0,0,0,9,0,1,Лекція
1,Понеділок,15:05,2,3,3,0,0,4,1,0,0,0,14,0,1,Лекція
2,Субота,18:10,3,4,4,1,0,-7,0,1,0,0,2,1,0,Лекція
3,Пʼятниця,8:30,5,1,4,0,0,34,0,0,1,0,0,0,0,Лабораторна
4,Пʼятниця,10:10,4,1,2,0,0,13,1,0,1,0,10,0,0,Лекція


## 4. Генерація погодних даних

Погодні дані вже враховані у попередньому коді (дощ, температура), але можна додати ще, якщо потрібно.

## 5. Генерація подій та свят

Дані про події (свята, карантин) вже враховані у полі "Подія". Можна деталізувати, якщо потрібно.

## 6. Генерація розкладу пар та типів занять

Розклад, тип пари вже враховані у попередньому коді.

## 7. Формування цільової змінної (відвідуваність)

Задамо правила для формування цільової змінної (кількість студентів на парі) на основі факторів.

In [4]:
# Формування цільової змінної (кількість студентів)
def calc_attendance(row):
    base = 30
    # Важливість предмету
    base += (row['Важливість_у_навчанні'] - 3) * 2
    base += (row['Важливість_у_житті'] - 3) * 1.5
    # Строгість викладача
    base += (row['Строгість_викладача'] - 3) * 2
    # Відмічають присутність
    if row['Відмічають_присутність']:
        base += 3
    # Дощ
    if row['Дощ']:
        base -= 3
    # Температура
    if row['Температура'] < 0 or row['Температура'] > 30:
        base -= 2
    # Контрольна/залік
    if row['Контрольна_або_залік']:
        base += 7
    # Вихідний напередодні
    if row['Вихідний_напередодні']:
        base -= 2
    # Вихідний завтра
    if row['Вихідний_завтра']:
        base -= 1
    # Подія (свято, карантин)
    if row['Подія']:
        base -= 10
    # Наближення сесії
    if row['Тижнів_до_екзамену'] < 2:
        base += 4
    # Попередня пара
    if not row['Попередня_пара']:
        base += 1
    # Наступна пара
    if not row['Наступна_пара']:
        base += 1
    # Тип пари
    if row['Тип_пари'] == 'Лекція':
        base -= 4
    elif row['Тип_пари'] == 'Лабораторна':
        base += 2
    # Шум
    base += np.random.normal(0, 2)
    return max(0, min(40, int(round(base))))

df['Відвідуваність'] = df.apply(calc_attendance, axis=1)
df.head()

,День_тижня,Час_пари,Важливість_у_навчанні,Важливість_у_житті,Строгість_викладача,Відмічають_присутність,Дощ,Температура,Контрольна_або_залік,Вихідний_напередодні,Вихідний_завтра,Подія,Тижнів_до_екзамену,Попередня_пара,Наступна_пара,Тип_пари,Відвідуваність
0,Понеділок,18:10,4,5,5,1,1,30,0,0,0,0,9,0,1,Лекція,35
1,Понеділок,15:05,2,3,3,0,0,4,1,0,0,0,14,0,1,Лекція,31
2,Субота,18:10,3,4,4,1,0,-7,0,1,0,0,2,1,0,Лекція,28
3,Пʼятниця,8:30,5,1,4,0,0,34,0,0,1,0,0,0,0,Лабораторна,39
4,Пʼятниця,10:10,4,1,2,0,0,13,1,0,1,0,10,0,0,Лекція,29


## 8. Об'єднання всіх факторів у фінальний датасет

Всі фактори вже об'єднані у DataFrame `df`. Переглянемо кілька перших рядків.

In [5]:
df.head(10)

,День_тижня,Час_пари,Важливість_у_навчанні,Важливість_у_житті,Строгість_викладача,Відмічають_присутність,Дощ,Температура,Контрольна_або_залік,Вихідний_напередодні,Вихідний_завтра,Подія,Тижнів_до_екзамену,Попередня_пара,Наступна_пара,Тип_пари,Відвідуваність
0,Понеділок,18:10,4,5,5,1,1,30,0,0,0,0,9,0,1,Лекція,35
1,Понеділок,15:05,2,3,3,0,0,4,1,0,0,0,14,0,1,Лекція,31
2,Субота,18:10,3,4,4,1,0,-7,0,1,0,0,2,1,0,Лекція,28
3,Пʼятниця,8:30,5,1,4,0,0,34,0,0,1,0,0,0,0,Лабораторна,39
4,Пʼятниця,10:10,4,1,2,0,0,13,1,0,1,0,10,0,0,Лекція,29
5,Четвер,13:30,4,3,4,0,1,23,1,1,0,0,5,0,1,Лабораторна,36
6,Понеділок,10:10,4,4,3,1,0,11,0,0,1,0,11,1,0,Лекція,36
7,Середа,15:05,3,5,1,1,0,4,0,1,1,0,11,1,1,Лекція,27
8,Понеділок,10:10,5,2,1,0,1,-5,1,1,1,0,6,0,0,Лабораторна,31
9,Субота,19:40,2,1,1,1,1,31,1,1,0,0,11,1,1,Лекція,20


## 9. Збереження датасету у файл

Збережемо фінальний датасет у форматі CSV для подальшого використання.

In [6]:
df.to_csv('synthetic_student_attendance.csv', index=False)
print('Датасет збережено у файл synthetic_student_attendance.csv')

Датасет збережено у файл synthetic_student_attendance.csv
